In [19]:
bayern_raw = readcsv("Bayern Munich Player Stats.csv");
real_raw = readcsv("Real Madrid Player Stats.csv");
juv_raw = readcsv("Juventus Player Stats.csv");
barc_raw = readcsv("Barcelona Player Stats.csv");

(m,n) = size(bayern_raw)
println(size(bayern_raw))
(m2,n2) = size(real_raw)
println(size(real_raw))
println(size(juv_raw))
println(size(barc_raw))

(26,30)
(26,30)
(27,30)
(27,30)


In [11]:
real_squad_table = real_raw
bayern_squad_table = bayern_raw;
juv_squad_table = juv_raw;
barc_squad_table = barc_raw;

squad_stat_values = real_squad_table[(2:26),(4:30)];
bay_stat_values = bayern_squad_table[(2:26),(4:30)];
juv_stat_values = juv_squad_table[(2:26),(4:30)];
barc_stat_values = barc_squad_table[(2:26),(4:30)];

In [26]:
using JuMP, Mosek
#Pre-generate teams
match_offense = Array{Float64}(zeros(4))
match_defense = Array{Float64}(zeros(4))
match_team_array = Array{Int64}(zeros(3,25,4))
for g in 1:3
    stat_values = Array{Int64}(zeros(25,27));
    squad_table = real_squad_table;
    season = Model(solver=MosekSolver(LOG=0))
    if g == 1
        stat_values = bay_stat_values;
        squad_table = bayern_squad_table;
    end
    if g == 2
        stat_values = juv_stat_values;
        squad_table = juv_squad_table;
    end
    if g == 3
        stat_values = barc_stat_values;
        squad_table = barc_squad_table;
    end
    gk = Array{Int64}(zeros(27))
    gk[9] = 1
    gk[10] = 1
    gk[11] = 1

    def = Array{Int64}(zeros(27))
    def[2] = 1
    def[12] = 1
    def[13] = 1
    def[16] = 1
    def[21] = 1
    def[23] = 1
    def[25] = 1

    str = Array{Int64}(zeros(27))
    str[1] = 1
    str[3] = 1
    str[5] = 1
    str[7] = 1
    str[8] = 1
    str[12] = 1
    str[15] = 1
    str[17] = 1
    str[18] = 1
    str[20] = 1
    str[22] = 1
    str[24] = 1
    str[27] = 1

    md_def = Array{Int64}(zeros(27))
    md_def[4] = 1
    md_def[6] = 1
    md_off = Array{Int64}(zeros(27))
    md_off[1] = 1
    md_off[3] = 1
    md_off[4] = 1
    md_off[5] = 1
    md_off[7] = 1
    md_off[8] = 1
    md_off[14] = 1
    md_off[15] = 1
    md_off[17] = 1
    md_off[18] = 1
    md_off[19] = 1
    md_off[20] = 1
    md_off[26] = 1
    gk_vals = Array{Int64}(zeros(25,27))
    for i in 1:25
        gk_vals[i,:] = stat_values[i,:].*gk
    end
    def_vals = Array{Int64}(zeros(25,27))
    for i in 1:25
        def_vals[i,:] = stat_values[i,:].*def
    end
    str_vals = Array{Int64}(zeros(25,27))
    for i in 1:25
        str_vals[i,:] = stat_values[i,:].*str
    end
    md_def_vals = Array{Int64}(zeros(25,27))
    for i in 1:25
        md_def_vals[i,:] = stat_values[i,:].*md_def
    end
    md_off_vals = Array{Int64}(zeros(25,27))
    for i in 1:25
        md_off_vals[i,:] = stat_values[i,:].*md_off
    end
    md_penalty = 1
    #Positions are goalkeeper, defence, midfield, and striker
    #There will be two sides, offense and defense
    #GK, defence, and midfield contribute to defense
    #Midfield and strikers contribute to offense
    #i is the game
    #j is the amount of players
    #k is the assignment of that player
    @variable(season, team[1:1,1:25,1:4], Bin)
    @variable(season, offense[1:1])
    @variable(season, defense[1:1])
    for i in 1:1
        for j in 1:25
            @constraint(season, sum(team[i,j,:]) <= 1)
        end
    end
    for i in 1:1
        #11 players on a team
        @constraint(season, sum(team[i,:,:]) == 11)
        #Need at least one GK
        @constraint(season, sum(team[i,:,1]) == 1)
        #Must have some defenders and strikers
        @constraint(season, sum(team[i,:,2]) >= 3)
        @constraint(season, sum(team[i,:,3]) >= 1)
        @constraint(season, sum(team[i,:,4]) >= 2)
        @constraint(season, defense[i] == sum(team[i,:,1]'*gk_vals) + sum(team[i,:,2]'*def_vals) + md_penalty*sum(team[i,:,4]'*md_def_vals))
        @constraint(season, offense[i] == sum(team[i,:,3]'*str_vals) + md_penalty*sum(team[i,:,4]'*md_off_vals))
    end
    @objective(season, Max, sum(offense) + sum(defense))

    status = solve(season)
    println(status)
    season_comps = getvalue(team)
    offenses = getvalue(offense)
    defenses = getvalue(defense)
    match_offense[g] = offenses[1];
    match_defense[g] = defenses[1];
    
    for i in 1:1
    println("Game ", i, " composition")
    println("----- Goalkeeper ------")
    for j in 1:25
        if season_comps[i,j,1] > 0
            println(squad_table[j+1,1])
        end
    end
    println("----- Defenders ------")
    for j in 1:25
        if season_comps[i,j,2] > 0
            println(squad_table[j+1,1])
        end
    end
    println("----- Strikers ------")
    for j in 1:25
        if season_comps[i,j,3] > 0
            println(squad_table[j+1,1])
        end
    end
    println("----- Midfielders ------")
    for j in 1:25
        if season_comps[i,j,4] > 0
            println(squad_table[j+1,1])
        end
    end
end
    println("Offense score is ", match_offense[g])
    println("Defense score is ", match_defense[g])
end


Optimal
Game 1 composition
----- Goalkeeper ------
Manuel Neuer
----- Defenders ------
Mats Hummels
Jerome Boateng
Javi Martinez
Arturo Vidal
----- Strikers ------
Robert Lewandowski
----- Midfielders ------
David Alaba
Thiago Alcantara
Franck Ribery
Arjen Robben
Douglas Costa
Arturo Vidal
Offense score is 7541.0
Defense score is 3057.0
Optimal
Game 1 composition
----- Goalkeeper ------
Gianluigi Buffon
----- Defenders ------
Giorgio Chiellini
Andrea Barzagli
Leonardo Bonucci
----- Strikers ------
Gonzalo Higuain
Mario Mandzukic
----- Midfielders ------
Dani Alves
Miralem Pjanic
Claudio Marchisio
Kwadwo Asamoah
Juan Cuadrado
Paulo Dybala
Offense score is 7337.0
Defense score is 3028.0
Optimal
Game 1 composition
----- Goalkeeper ------
Marc-Andre Ter Stegen
----- Defenders ------
Gerard Pique
Javier Mascherano
Jeremy Mathieu
----- Strikers ------
Luis Suarez
----- Midfielders ------
Jordi Alba
Ivan Rakitic
Arda Turan
Andres Iniesta
Rafinha
Lionel Messi
Neymar
Offense score is 7644.00000

In [6]:
using JuMP
rm = Model()

squad_stat_values = real_squad_table[(2:26),(4:30)]
bay_stat_values = bayern_squad_table[(2:26),(4:30)]
juv_stat_values = juv_squad_table[(2:26),(4:30)]
barc_stat_values = barc_squad_table[(2:26),(4:30)]

@variable(rm, player_names[1:25])
@variable(rm, position[1:25])
@variable(rm, finalteam)

@variable(rm, gk_number)
@variable(rm, def_number)
@variable(rm, mid_number)
@variable(rm, fwd_number)
@variable(rm, count)

@constraint(rm, count<=11)

gk_number = 1
def_number = 4
mid_number = 4
fwd_number = 2
count = 1;

finalteam = Array{Any,2}(zeros(11,29))
indexes = Array{Int64}(zeros(11))

#-----------------------------------------------------------------------------------------------------------------------------
#storing player attributes

player_names = Array{String}(real_squad_table[2:26,1])
position = Array{String}(real_squad_table[2:26,3])
acceleration = Array{Int64}(real_squad_table[2:26,4])
aggression = Array{Int64}(real_squad_table[2:26,5])
agility = Array{Int64}(real_squad_table[2:26,6])
balance = Array{Int64}(real_squad_table[2:26,7])
ball_control = Array{Int64}(real_squad_table[2:26,8])
crossing = Array{Int64}(real_squad_table[2:26,9])
dribbling = Array{Int64}(real_squad_table[2:26,10])
finishing = Array{Int64}(real_squad_table[2:26,11])
gk_diving = Array{Int64}(real_squad_table[2:26,12])
gk_handling = Array{Int64}(real_squad_table[2:26,13])
gk_reflexes = Array{Int64}(real_squad_table[2:26,14])
heading_accuracy = Array{Int64}(real_squad_table[2:26,15])
interceptions = Array{Int64}(real_squad_table[2:26,16])
long_passing = Array{Int64}(real_squad_table[2:26,17])
long_shots = Array{Int64}(real_squad_table[2:26,18])
marking = Array{Int64}(real_squad_table[2:26,19])
positioning = Array{Int64}(real_squad_table[2:26,20])
reactions = Array{Int64}(real_squad_table[2:26,21])
short_passing = Array{Int64}(real_squad_table[2:26,22])
shotpower = Array{Int64}(real_squad_table[2:26,23])
sliding_tackle = Array{Int64}(real_squad_table[2:26,24])
sprint_speed = Array{Int64}(real_squad_table[2:26,25])
standing_tackle = Array{Int64}(real_squad_table[2:26,26])
stamina = Array{Int64}(real_squad_table[2:26,27])
strength = Array{Int64}(real_squad_table[2:26,28])
vision = Array{Int64}(real_squad_table[2:26,29])
volleys = Array{Int64}(real_squad_table[2:26,30])



#-----------------------------------------------------------------------------------------------------------------------------
#choosing goalkeeper
max = 0;
goalkeeper_score = 0;

for i = 1:length(player_names)
    goalkeeper_score = squad_stat_values[i,9] + squad_stat_values[i,10] + squad_stat_values[i,11]
    if goalkeeper_score>=max
        max = goalkeeper_score
        indexes[count] = i
    end
end

if count<=10
    count = count + 1;
end

#-----------------------------------------------------------------------------------------------------------------------------
#choosing defence
defender_scores = Array{Int64}(zeros(1,25))[:]

for i = 1:25
    defender_scores[i] = squad_stat_values[i,2] + squad_stat_values[i,12] + squad_stat_values[i,13] + squad_stat_values[i,16] 
    + squad_stat_values[i,21] + squad_stat_values[i,23] + squad_stat_values[i,25]
end

defender_index = sortperm(defender_scores, rev=true)
i = 1;
def_count = 0;

while def_count<=def_number-0.1
    if position[defender_index[i]] == "CB" || position[defender_index[i]] == "FB"
        indexes[count] = defender_index[i]
        def_count = def_count + 1;
        if count<=10
            count = count + 1;
        end
    end
    i=i+1
end
#-----------------------------------------------------------------------------------------------------------------------------
#choosing midfield
midfielder_scores = Array{Int64}(zeros(1,25))[:]

for i = 1:25
    midfielder_scores[i] = squad_stat_values[i,1] + squad_stat_values[i,3] + squad_stat_values[i,4] + squad_stat_values[i,5] + squad_stat_values[i,6] + squad_stat_values[i,7] + squad_stat_values[i,8] + squad_stat_values[i,14] + squad_stat_values[i,15] + squad_stat_values[i,17] + squad_stat_values[i,18] + squad_stat_values[i,19] + squad_stat_values[i,20] + squad_stat_values[i,26]
end

midfielder_index = sortperm(midfielder_scores, rev=true)
j = 1;
mid_count = 0;

while mid_count<=mid_number-0.1
    if position[midfielder_index[j]] == "CM" || position[midfielder_index[j]] == "WG"
        indexes[count] = midfielder_index[j]
        if count<=10
            count = count + 1;
        end
        mid_count = mid_count + 1
    end
    j=j+1
end
#-----------------------------------------------------------------------------------------------------------------------------
#choosing strikers
forward_scores = Array{Int64}(zeros(1,25))[:]

for i = 1:25
    forward_scores[i] = squad_stat_values[i,1] + squad_stat_values[i,3] + squad_stat_values[i,5] + squad_stat_values[i,7] + squad_stat_values[i,8] + squad_stat_values[i,12] + squad_stat_values[i,15] + squad_stat_values[i,17] + squad_stat_values[i,18] + squad_stat_values[i,20] + squad_stat_values[i,22] + squad_stat_values[i,24] + squad_stat_values[i,27]
end

forward_index = sortperm(forward_scores, rev=true)
i = 1
fwd_count = 0;

while fwd_count<=fwd_number-0.1
    if position[forward_index[i]] == "FW"
        indexes[count] = forward_index[i]
        if count<=10
            count = count + 1;
        end
        fwd_count = fwd_count+1
    end
    i=i+1
end
#-----------------------------------------------------------------------------------------------------------------------------
#assembling team
for i = 1:11
    finalteam[i,1] = player_names[indexes[i]]
    finalteam[i,2] = position[indexes[i]]
    finalteam[i,3] = acceleration[indexes[i]]
    finalteam[i,4] = aggression[indexes[i]]
    finalteam[i,5] = agility[indexes[i]]
    finalteam[i,6] = balance[indexes[i]]
    finalteam[i,7] = ball_control[indexes[i]]
    finalteam[i,8] = crossing[indexes[i]]
    finalteam[i,9] = dribbling[indexes[i]]
    finalteam[i,10] = finishing[indexes[i]]
    finalteam[i,11] = gk_diving[indexes[i]]
    finalteam[i,12] = gk_handling[indexes[i]]
    finalteam[i,13] = gk_reflexes[indexes[i]]
    finalteam[i,14] = heading_accuracy[indexes[i]]
    finalteam[i,15] = interceptions[indexes[i]]
    finalteam[i,16] = long_passing[indexes[i]]
    finalteam[i,17] = long_shots[indexes[i]]
    finalteam[i,18] = marking[indexes[i]]
    finalteam[i,19] = positioning[indexes[i]]
    finalteam[i,20] = reactions[indexes[i]]
    finalteam[i,21] = short_passing[indexes[i]]
    finalteam[i,22] = shotpower[indexes[i]]
    finalteam[i,23] = sliding_tackle[indexes[i]]
    finalteam[i,24] = sprint_speed[indexes[i]]
    finalteam[i,25] = standing_tackle[indexes[i]]
    finalteam[i,26] = stamina[indexes[i]]
    finalteam[i,27] = strength[indexes[i]]
    finalteam[i,28] = vision[indexes[i]]
    finalteam[i,29] = volleys[indexes[i]]
end

println("Real Madrid's team is: ")
finalteam[:,:]

Real Madrid's team is: 


11×29 Array{Any,2}:
 "Keylor Navas"       "GK"  61  32  70  …  30  21  14  52  14  41  65  54  11
 "Pepe"               "CB"  65  94  63     67  63  90  75  89  64  84  48  23
 "Sergio Ramos"       "CB"  77  84  80     76  79  90  78  89  84  81  63  66
 "Raphael Varane"     "CB"  76  73  58     72  52  85  78  86  70  84  56  42
 "Marcelo"            "FB"  77  84  77     81  83  87  80  84  82  77  75  54
 "Cristiano Ronaldo"  "WG"  91  63  90  …  82  94  23  92  31  92  80  85  88
 "Gareth Bale"        "WG"  93  65  77     86  91  52  95  55  78  80  79  76
 "Luka Modric"        "CM"  77  62  93     92  73  73  71  80  83  58  90  74
 "James Rodriguez"    "CM"  77  65  82     84  86  30  75  42  68  70  88  91
 "Karim Benzema"      "FW"  79  65  77     86  83  12  82  15  74  78  85  79
 "Alvaro Morata"      "FW"  79  54  59  …  74  78  20  86  14  68  82  76  80